In [ ]:
# ==========================================================
# CuPy Reservoir Simulation: NARMA10 Prediction Task (GPU)
# ==========================================================
# Ensure you have a GPU runtime selected in Colab
# Install CuPy (choose the version matching your Colab CUDA version, e.g., 11.x or 12.x)
# !pip install cupy-cuda11x
# Or: !pip install cupy-cuda12x
# Check CUDA version: !nvcc --version

import cupy as cp # Use CuPy for GPU arrays and operations
import numpy as np # Use NumPy for CPU things (NARMA gen, plotting, sklearn)
import matplotlib.pyplot as plt
import time

# --- Imports for Readout Training & Evaluation ---
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

# --- Physical Constants ---
GAMMA_LL = 2.2128e5  # Gyromagnetic ratio (rad/(T*s) or m/(A*s))
ALPHA = 0.05       # Gilbert damping parameter
MS = 4.8e5         # Saturation magnetization (A/m)
KB = 1.380649e-23  # Boltzmann constant (J/K)
MU0 = 4 * cp.pi * 1e-7 # Permeability of free space (T*m/A or H/m) - Use cp.pi
T_TEMP = 300       # Temperature (K) for thermal noise

# --- Reservoir Parameters ---
N_MAGNETOSOMES = 100     # Number of magnetosomes
RESERVOIR_SIZE = 500e-9 # Size of the cubic area (meters)

# Magnetosome properties
DIAMETER = 45e-9
RADIUS = DIAMETER / 2
VOLUME = (4/3) * np.pi * RADIUS**3 # Use np.pi for scalar calculation
KU = 1.5e4 # Anisotropy constant (J/m^3) - might need tuning
EASY_AXIS_NP = np.array([0., 0., 1.], dtype=np.float64)

# --- Simulation Time ---
N_STEPS_NARMA = 2000 # Total length of NARMA sequence
WASHOUT_STEPS = 200  # Discard initial steps before training readout
T_STEP_NARMA = 50e-12 # Time duration of each NARMA input step (e.g., 50 ps)
T_TOTAL = N_STEPS_NARMA * T_STEP_NARMA

# --- Euler-Maruyama Time Step ---
# IMPORTANT: Needs to be very small for stability! May require tuning.
DT_SIM = 1e-15 # Simulation time step (s)
N_SIM_STEPS = int(T_TOTAL / DT_SIM)
# Saving interval to manage memory (save every N_SAVE_DIV steps)
N_SAVE_DIV = max(1, int(T_STEP_NARMA / DT_SIM)) # Save approx once per NARMA step
N_SAVES = N_SIM_STEPS // N_SAVE_DIV

print(f"Simulation Parameters:")
print(f"  N Particles: {N_MAGNETOSOMES}")
print(f"  Total Time: {T_TOTAL:.2e} s")
print(f"  Sim Time Step (dt): {DT_SIM:.1e} s")
print(f"  Total Sim Steps: {N_SIM_STEPS}")
print(f"  Saving data points: {N_SAVES}")


# --- Input Scaling ---
INPUT_SCALE_H = 5e3 # Scaling factor for NARMA input into H_ext (A/m)

# ==========================================================
# --- Reservoir Structure Setup (using CuPy) ---
# ==========================================================

def create_reservoir_cupy(n_magnetosomes, size, vol, Ku_val, Ms_val, easy_axis_np):
    """ Creates initial positions and M vectors using NumPy/CuPy """
    print(f"Generating {n_magnetosomes} random magnetosome positions...")
    # Generate positions on CPU first
    positions_np = (np.random.rand(n_magnetosomes, 3) - 0.5) * size
    positions_gpu = cp.asarray(positions_np, dtype=cp.float64)

    # Initial magnetization (random small tilt from easy axis)
    random_tilt_np = (np.random.rand(n_magnetosomes, 3) - 0.5) * 0.2
    initial_M_dir_np = easy_axis_np + random_tilt_np
    norm_np = np.linalg.norm(initial_M_dir_np, axis=1, keepdims=True)
    initial_M_dir_norm_np = initial_M_dir_np / np.where(norm_np > 1e-9, norm_np, 1.0)
    initial_M_np = initial_M_dir_norm_np * Ms_val
    initial_M_gpu = cp.asarray(initial_M_np, dtype=cp.float64)

    # Static parameters as CuPy arrays
    V_gpu = cp.full(n_magnetosomes, vol, dtype=cp.float64)
    Ku_gpu = cp.full(n_magnetosomes, Ku_val, dtype=cp.float64)
    Ms_gpu = cp.full(n_magnetosomes, Ms_val, dtype=cp.float64)
    easy_axis_gpu = cp.tile(cp.asarray(easy_axis_np, dtype=cp.float64), (n_magnetosomes, 1))

    print(f"Generated reservoir with {n_magnetosomes} magnetosomes.")
    return positions_gpu, initial_M_gpu, V_gpu, Ku_gpu, Ms_gpu, easy_axis_gpu

# Create reservoir - initial state and static parameters
pos_gpu, M_initial_gpu, V_gpu, Ku_gpu, Ms_gpu, easy_axis_gpu = \
    create_reservoir_cupy(N_MAGNETOSOMES, RESERVOIR_SIZE, VOLUME, KU, MS, EASY_AXIS_NP)

# ==========================================================
# --- NARMA10 Sequence Generation (NumPy) ---
# ==========================================================
def generate_narma10_sequence(n_steps):
    # (Same function as before)
    print(f"Generating NARMA10 sequence of length {n_steps}...")
    u = np.random.rand(n_steps) * 0.5
    y = np.zeros(n_steps)
    for k in range(9, n_steps - 1):
        y[k+1] = (0.3 * y[k] + 0.05 * y[k] * np.sum(y[k-9:k+1]) +
                    1.5 * u[k-9] * u[k] + 0.1)
    if np.any(np.isnan(y)) or np.any(np.isinf(y)):
        print("Warning: NaN or Inf encountered in NARMA10 sequence, replacing with 0.")
        y = np.nan_to_num(y)
    print("NARMA10 sequence generated.")
    return u, y

u_narma, y_narma_target = generate_narma10_sequence(N_STEPS_NARMA)
# Precompute H_ext values (on CPU first, then maybe move to GPU if needed)
H_ext_values_np = np.zeros((N_STEPS_NARMA, 3), dtype=np.float64)
H_ext_values_np[:, 1] = INPUT_SCALE_H * u_narma # Modulate Hy
H_ext_values_gpu = cp.asarray(H_ext_values_np) # Move to GPU

# Function to get H_ext at time t (using CuPy/NumPy)
def get_H_ext_t_cupy(t, H_ext_vals_gpu, t_step_narma):
    # Calculate index based on NARMA step duration
    idx = min(int(np.floor(t / t_step_narma)), H_ext_vals_gpu.shape[0] - 1)
    # Retrieve value from GPU array
    return H_ext_vals_gpu[idx]

# ==========================================================
# --- Field Calculations (CuPy versions) ---
# ==========================================================

def calculate_anisotropy_field_cupy(M, Ku, Ms, easy_axis):
    # M shape (N, 3), Ku shape (N,), Ms shape (N,), easy_axis shape (N, 3)
    M_dot_easy_axis = cp.sum(M * easy_axis, axis=1, keepdims=True) # Shape (N, 1)
    H_anis_magnitude = (2.0 * Ku / (MU0 * Ms))[:, None] # Shape (N, 1)
    # Ensure Ms is not zero for division
    safe_Ms = cp.where(Ms > 1e-12, Ms, 1.0)[:, None]
    H_anis = H_anis_magnitude * M_dot_easy_axis / safe_Ms * easy_axis
    return H_anis

def calculate_dipolar_field_cupy(M, V, pos):
    # M shape (N, 3), V shape (N,), pos shape (N, 3)
    N = M.shape[0]
    if N <= 1: return cp.zeros_like(M)

    # Use broadcasting to calculate pairwise vectors and distances efficiently
    pos_i = pos[:, cp.newaxis, :] # Shape (N, 1, 3)
    pos_j = pos[cp.newaxis, :, :] # Shape (1, N, 3)
    r_vec_ij = pos_i - pos_j      # Shape (N, N, 3)

    r_dist_sq = cp.sum(r_vec_ij**2, axis=2) # Shape (N, N)
    cp.fill_diagonal(r_dist_sq, cp.inf) # Avoid self-interaction / div by zero
    r_dist = cp.sqrt(r_dist_sq) # Shape (N, N)

    # Calculate r_hat safely by handling denominator first
    r_dist_denom = cp.where(r_dist > 1e-15, r_dist, 1.0) # Replace small distances with 1
    r_hat_ij_raw = r_vec_ij / r_dist_denom[..., None] # Perform division
    # Zero out entries where distance was originally too small
    r_hat_ij = cp.where(r_dist[..., None] > 1e-15, r_hat_ij_raw, 0.0)

    # Magnetic moments mj = Mj * Vj
    M_j = M[cp.newaxis, :, :] # Shape (1, N, 3)
    V_j = V[cp.newaxis, :, None] # Shape (1, N, 1)
    m_j = M_j * V_j # Shape (1, N, 3)

    # Dipolar terms: 3(mj . r_hat_ij)r_hat_ij - mj
    dot_prod = cp.sum(m_j * r_hat_ij, axis=2) # Shape (N, N)
    term1 = 3.0 * dot_prod[..., None] * r_hat_ij # Shape (N, N, 3)
    term2 = m_j # Shape (1, N, 3) -> broadcasts to (N, N, 3)
    field_term = term1 - term2 # Shape (N, N, 3)

    # --- MODIFIED inv_r3 calculation ---
    # Prefactor 1 / (4 * pi * r_dist^3)
    r_dist_cubed = r_dist**3 # Shape (N, N)
    # Create a safe denominator: replace small/zero distances with infinity
    safe_r_dist_cubed = cp.where(r_dist > 1e-15, r_dist_cubed, cp.inf)
    # Perform division (1.0 / infinity becomes 0.0)
    inv_r3 = 1.0 / safe_r_dist_cubed
    # --- End of modification ---

    prefactor = inv_r3 / (4 * cp.pi) # Shape (N, N)

    # Sum contributions over j for each i
    H_dip_contributions = prefactor[..., None] * field_term # Shape (N, N, 3)
    H_dip = cp.sum(H_dip_contributions, axis=1) # Sum over j -> shape (N, 3)
    return H_dip

def calculate_thermal_field_cupy(V, Ms, temperature, dt):
    # V shape (N,), Ms shape (N,)
    N = V.shape[0]
    if temperature <= 0 or dt <= 0: return cp.zeros((N, 3), dtype=cp.float64)

    # Strength factor - shape (N,)
    thermal_strength_factor = cp.sqrt(
        (2 * ALPHA * KB * temperature) /
        (GAMMA_LL * MU0 * Ms * V * dt) # dt is crucial here
    )
    # Random Gaussian numbers from CuPy - shape (N, 3)
    random_gaussian = cp.random.normal(0.0, 1.0, (N, 3), dtype=cp.float64)
    H_th = thermal_strength_factor[:, None] * random_gaussian # Shape (N, 3)
    return H_th

def normalize_M_cupy(M, Ms):
    """ Normalizes M vectors to magnitude Ms """
    # M shape (N, 3), Ms shape (N,)
    norm_M = cp.linalg.norm(M, axis=1, keepdims=True) # Shape (N, 1)
    # Avoid division by zero
    safe_norm = cp.where(norm_M > 1e-12, norm_M, 1.0)
    M_normalized = M / safe_norm * Ms[:, None] # Multiply by Ms column vector
    # Handle cases where initial norm was zero (or very small) - keep them zero
    M_final = cp.where(norm_M > 1e-12, M_normalized, M)
    return M_final

# ==========================================================
# --- Euler-Maruyama Simulation Loop ---
# ==========================================================

print(f"\n--- Running CuPy Euler-Maruyama simulation ---")
start_sim_time = time.time()

# Initialize M state on GPU
M_gpu = cp.copy(M_initial_gpu)

# --- Allocate memory on GPU for storing results periodically ---
saved_states_gpu = cp.zeros((N_SAVES, N_MAGNETOSOMES * 3), dtype=cp.float64)
saved_times_np = np.zeros(N_SAVES) # Store times on CPU
save_count = 0

# --- Main Loop ---
for step in range(N_SIM_STEPS):
    current_time = step * DT_SIM

    # 1. Calculate Effective Field
    H_ext_t = get_H_ext_t_cupy(current_time, H_ext_values_gpu, T_STEP_NARMA)
    H_ext_t_tiled = cp.tile(H_ext_t, (N_MAGNETOSOMES, 1)) # Tile to (N, 3)

    H_anis = calculate_anisotropy_field_cupy(M_gpu, Ku_gpu, Ms_gpu, easy_axis_gpu)
    H_dip = calculate_dipolar_field_cupy(M_gpu, V_gpu, pos_gpu)
    H_th = calculate_thermal_field_cupy(V_gpu, Ms_gpu, T_TEMP, DT_SIM)

    H_eff = H_ext_t_tiled + H_anis + H_dip + H_th

    # 2. Calculate LLG Update (Euler-Maruyama step)
    prefactor1 = -GAMMA_LL / (1 + ALPHA**2)
    safe_Ms_gpu = cp.where(Ms_gpu > 1e-12, Ms_gpu, 1.0)[:, None] # Shape (N, 1)
    prefactor2 = prefactor1 * ALPHA / safe_Ms_gpu

    M_cross_Heff = cp.cross(M_gpu, H_eff)
    M_cross_M_cross_Heff = cp.cross(M_gpu, cp.cross(M_gpu, H_eff)) # Recompute for safety

    dM = (prefactor1 * M_cross_Heff + prefactor2 * M_cross_M_cross_Heff) * DT_SIM

    # 3. Update M
    M_gpu += dM

    # 4. Normalize M
    M_gpu = normalize_M_cupy(M_gpu, Ms_gpu)

    # 5. Save State Periodically
    if (step + 1) % N_SAVE_DIV == 0:
        if save_count < N_SAVES:
            saved_states_gpu[save_count, :] = M_gpu.flatten()
            saved_times_np[save_count] = (step + 1) * DT_SIM # Save end time of interval
            save_count += 1

# Ensure all GPU operations are finished before stopping timer
cp.cuda.stream.get_current_stream().synchronize()
end_sim_time = time.time()
print(f"Simulation finished in {end_sim_time - start_sim_time:.2f} seconds.")

# --- Post-process saved states ---
saved_indices = np.searchsorted(saved_times_np, T_EVAL[1:], side='left')
saved_indices = np.clip(saved_indices, 0, N_SAVES - 1)
if N_SAVES == N_STEPS_NARMA:
    print("Using saved states directly for readout.")
    reservoir_states_flat_gpu = saved_states_gpu
    sim_times_readout_gpu = cp.asarray(saved_times_np)
else:
    print(f"Warning: Number of saves ({N_SAVES}) doesn't match NARMA steps ({N_STEPS_NARMA}). Selecting closest points.")
    reservoir_states_flat_gpu = saved_states_gpu[saved_indices, :]
    sim_times_readout_gpu = cp.asarray(saved_times_np[saved_indices])
    if len(y_narma_target) != reservoir_states_flat_gpu.shape[0]:
         print("Adjusting target length to match selected states.")
         y_narma_target = y_narma_target[:reservoir_states_flat_gpu.shape[0]]

print(f"Selected output states shape: {reservoir_states_flat_gpu.shape}")

# ==========================================================
# --- Readout Training & Evaluation ---
# ==========================================================
# (This part remains the same)
print("\n--- Training Readout for NARMA10 Prediction ---")
# 1. Prepare Data
print("Transferring states to CPU...")
X_reservoir_cpu = cp.asnumpy(reservoir_states_flat_gpu)
y_target_cpu = y_narma_target
sim_times_readout = cp.asnumpy(sim_times_readout_gpu)

if washout_steps > 0 and washout_steps < X_reservoir_cpu.shape[0]:
    print(f"Applying washout: discarding first {washout_steps} steps.")
    washout_time_actual = washout_steps * T_STEP_NARMA
    washout_idx_readout = np.searchsorted(sim_times_readout, washout_time_actual, side='left')
    if washout_idx_readout > 0 and washout_idx_readout < X_reservoir_cpu.shape[0]:
         print(f"  (Discarding {washout_idx_readout} saved time points)")
         X_readout = X_reservoir_cpu[washout_idx_readout:, :]
         y_readout_target = y_target_cpu[washout_idx_readout:]
         times_readout = sim_times_readout[washout_idx_readout:]
    else:
         print("  Washout index out of bounds, not applying.")
         X_readout = X_reservoir_cpu; y_readout_target = y_target_cpu
         times_readout = sim_times_readout
else:
    X_readout = X_reservoir_cpu; y_readout_target = y_target_cpu
    times_readout = sim_times_readout; print("Washout not applied.")

print(f"Readout training data shape - X: {X_readout.shape}, y: {y_readout_target.shape}")

if X_readout.shape[0] == 0 or y_readout_target.shape[0] == 0 or X_readout.shape[0] != y_readout_target.shape[0]:
    print("Error: Invalid shapes for readout training after washout/selection. Skipping readout.")
else:
    # 2. Feature Scaling
    print("Scaling features...")
    scaler = StandardScaler(); X_readout_scaled = scaler.fit_transform(X_readout)
    # 3. Train Ridge Regression Model
    readout_model = Ridge(alpha=10.0)
    print("Training Ridge regression model...")
    start_train_time = time.time(); readout_model.fit(X_readout_scaled, y_readout_target)
    end_train_time = time.time(); print(f"Training finished in {end_train_time - start_train_time:.3f} seconds.")
    # 4. Predict and Evaluate
    print("Predicting NARMA10 sequence using trained readout...")
    y_readout_predicted = readout_model.predict(X_readout_scaled)
    mse = mean_squared_error(y_readout_target, y_readout_predicted); rmse = np.sqrt(mse)
    target_std = np.std(y_readout_target); nrmse = rmse / target_std if target_std > 1e-9 else np.inf
    print(f"\n--- Evaluation ---"); print(f"Target Standard Deviation: {target_std:.4f}")
    print(f"Root Mean Square Error (RMSE): {rmse:.4f}"); print(f"Normalized RMSE (NRMSE): {nrmse:.4f}")
    # 5. Plot Results
    print("\nPlotting NARMA10 target vs. prediction...")
    plt.figure(figsize=(15, 5))
    plt.plot(times_readout, y_readout_target, label='Target NARMA10 Output', color='black', linewidth=1.5)
    plt.plot(times_readout, y_readout_predicted, label=f'Predicted Output (NRMSE={nrmse:.3f})', color='red', linestyle='--', linewidth=1.5)
    plt.xlabel('Time (s)'); plt.ylabel('NARMA10 Value')
    plt.title(f'NARMA10 Time Series Prediction (CuPy Euler - N={N_MAGNETOSOMES}, T={T_TEMP}K, dt={DT_SIM:.1e})')
    plt.legend(); plt.grid(True, linestyle='--', alpha=0.6)
    plt.ylim(min(y_readout_target.min(), y_readout_predicted.min()) - 0.1, max(y_readout_target.max(), y_readout_predicted.max()) + 0.1)
    plt.tight_layout(); plt.show()

print("\nCuPy NARMA10 simulation and evaluation complete.")

Simulation Parameters:
  N Particles: 100
  Total Time: 1.00e-07 s
  Sim Time Step (dt): 1.0e-15 s
  Total Sim Steps: 100000000
  Saving data points: 2000
Generating 100 random magnetosome positions...
Generated reservoir with 100 magnetosomes.
Generating NARMA10 sequence of length 2000...
NARMA10 sequence generated.

--- Running CuPy Euler-Maruyama simulation ---
